In [1]:
import pdfplumber

pdf_path = r"C:\Users\hp\PycharmProjects\Mini_Project\CreditcardscomInc_20070810_S-1_EX-10.33_362297_EX-10.33_Affiliate Agreement.pdf"

text = ""
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text += page.extract_text() + "\n"

pdf_content = text

In [2]:
import pytesseract
from pdf2image import convert_from_path

# Path to your PDF
pdf_path = r"C:\Users\hp\PycharmProjects\Mini_Project\CreditcardscomInc_20070810_S-1_EX-10.33_362297_EX-10.33_Affiliate Agreement.pdf"

# Tell pytesseract where Tesseract OCR is installed
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Convert PDF pages to images using Poppler
pages = convert_from_path(pdf_path, poppler_path=r"C:\Users\hp\Downloads\Release-25.07.0-0\poppler-25.07.0\Library\bin")

ocr_text = ""
for i, page in enumerate(pages):
    # Extract text from each page using OCR
    text = pytesseract.image_to_string(page)
    ocr_text += text + "\n"
    print(f"Processed page {i+1}/{len(pages)}")

# Print first 1000 characters of extracted text
print(ocr_text[:1000])


Processed page 1/12
Processed page 2/12
Processed page 3/12
Processed page 4/12
Processed page 5/12
Processed page 6/12
Processed page 7/12
Processed page 8/12
Processed page 9/12
Processed page 10/12
Processed page 11/12
Processed page 12/12
Exhibit 10.33

Last Updated: April 6, 2007

CHASE AFFILIATE AGREEMENT

THIS AGREEMENT sets forth the terms and conditions agreed to between Chase Bank USA, N.A. (?Chase?) and you as an “Affiliate” in the Chase
Affiliate Program (the “Affiliate Program”). Once accepted into the Affiliate Program, an Affiliate can establish links from the Affiliate’s Website to
[Chase.com]. Chase will pay Affiliate a fee for each approved credit card account that originates from a link in Affiliate’s Website.

THIS IS A LEGAL AND CONTRACTUALLY BINDING AGREEMENT BETWEEN AFFILIATE AND CHASE. TO APPLY TO THE AFFILIATE
PROGRAM, YOU MUST COMPLETE AND SUBMIT THE AFFILIATE REGISTRATION FORM AND CLICK ON THE “AGREE” BUTTON BELOW
TO INDICATE YOUR WILLINGNESS TO BE BOUND TO C

In [8]:
import difflib

# Take first 100000 characters to compare
pdf_snippet = pdf_content[:100000]
ocr_snippet = ocr_text[:100000]

# Calculate similarity ratio
similarity = difflib.SequenceMatcher(None, pdf_snippet, ocr_snippet).ratio()

print(f"Similarity: {similarity*100:.2f}%")


Similarity: 97.80%


In [16]:
#!pip install nltk


In [19]:
import sys
print(sys.executable)
#"C:\Users\hp\PycharmProjects\Mini_Project\.venv\Scripts\python.exe" -m pip install nltk

C:\Users\hp\PycharmProjects\Mini_Project\.venv\Scripts\python.exe


In [3]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:
import sys
#!{sys.executable} -m pip install faiss-cpu
!{sys.executable} -m pip install --upgrade sentencepiece transformers torch


In [9]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Load embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Split OCR text into chunks (paragraphs or 500-word blocks)
def split_text(text, max_words=200):
    words = text.split()
    return [" ".join(words[i:i+max_words]) for i in range(0, len(words), max_words)]

texts = split_text(ocr_text)

# Encode into embeddings
embeddings = model.encode(texts)        # shape: (num_chunks, dim)
embeddings = np.array(embeddings).astype("float32")

# Normalize embeddings for cosine similarity
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

# Build FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)    # inner product = cosine similarity (since normalized)
index.add(embeddings)

print("FAISS index is ready with", index.ntotal, "documents")


C:\Users\hp\PycharmProjects\mini\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


FAISS index is ready with 21 documents


In [10]:

# -----------------------------
# Example: Semantic search in a legal contract
# -----------------------------

# Assume you already have:
# texts -> list of clauses/sentences
# model -> SentenceTransformer model
# index -> FAISS index built from embeddings of texts

# Example query
query = "What is restricted content in this agreement?"

# Step 1: Encode query and normalize for cosine similarity
query_emb = model.encode([query], convert_to_numpy=True, normalize_embeddings=True)
query_emb = query_emb.astype("float32")

# Step 2: Search top-k matches in FAISS index
top_k = 3
D, I = index.search(query_emb, k=top_k)  # D: scores, I: indices of matches

# Step 3: Display results
print("\nQuery:", query)
for rank, idx in enumerate(I[0]):
    print(f"\nMatch {rank+1} (score={D[0][rank]:.4f}):")
    print(texts[idx])



Query: What is restricted content in this agreement?

Match 1 (score=0.4159):
Chase, its shareholders, officers, directors, employees, agents, affiliates and their respective directors, officers, employees and agents, successors and assigns, from and against any and all claims, demands, losses, liabilities, damages or expenses (including attorneys’ fees and costs) of any nature whatsoever incurred or suffered by Chase (collectively the “losses”), in so far as such losses (or actions in respect thereof) arise out of, are related to, or are based on (i) the breach of any representation, warranty, or covenant made by Affiliate herein; or (ii) any claim related to Affiliate’s site. 18. Confidentiality Except as otherwise provided in this Agreement or with the consent of the other party hereto, each of the parties hereto agrees that all information including, without limitation, the terms of this Agreement, business and financial information, customer and vendor lists, and pricing and sale

In [19]:

from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load T5 model (offline once downloaded, or cache)
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def simplify_text(text):
    """
    Simplify legal/complex text using T5 model
    """
    # T5 can be instructed with a "simplify:" prefix
    input_text = "simplify: " + text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    
    outputs = model.generate(inputs, max_length=512, num_beams=4, early_stopping=True)
    simplified = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return simplified


ImportError: 
T5Tokenizer requires the SentencePiece library but it was not found in your environment. Check out the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [ ]:



# 1️⃣ Imports
from transformers import T5Tokenizer, T5ForConditionalGeneration

# 2️⃣ Load the T5 model (pretrained)
model_name = "t5-small"  # lightweight T5 for simplification
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# 3️⃣ Function to simplify text
def simplify_text(text):
    input_text = "simplify: " + text  # T5 task prefix
    inputs = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model.generate(
        inputs, 
        max_length=150, 
        num_beams=4, 
        early_stopping=True
    )
    simplified = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return simplified

# 4️⃣ Example usage
# ocr_text = """
# To enroll in the Affiliate Program, you must submit a complete “Affiliate Registration Form” via the Chase Affiliate Website. 
# Chase will evaluate your registration form and may reject your site if it contains restricted content such as offensive, illegal, or sexually explicit material.
# """

simplified_version = simplify_text(ocr_text)
print("Original Text:\n", ocr_text)
print("\nSimplified Text:\n", simplified_version)


Original Text:
 Exhibit 10.33

Last Updated: April 6, 2007

CHASE AFFILIATE AGREEMENT

THIS AGREEMENT sets forth the terms and conditions agreed to between Chase Bank USA, N.A. (?Chase?) and you as an “Affiliate” in the Chase
Affiliate Program (the “Affiliate Program”). Once accepted into the Affiliate Program, an Affiliate can establish links from the Affiliate’s Website to
[Chase.com]. Chase will pay Affiliate a fee for each approved credit card account that originates from a link in Affiliate’s Website.

THIS IS A LEGAL AND CONTRACTUALLY BINDING AGREEMENT BETWEEN AFFILIATE AND CHASE. TO APPLY TO THE AFFILIATE
PROGRAM, YOU MUST COMPLETE AND SUBMIT THE AFFILIATE REGISTRATION FORM AND CLICK ON THE “AGREE” BUTTON BELOW
TO INDICATE YOUR WILLINGNESS TO BE BOUND TO CHASE BY THIS AGREEMENT. THIS AGREEMENT WILL TAKE EFFECT IF AND
WHEN CHASE REVIEWS AND ACCEPTS YOUR REGISTRATION FORM AND PROVIDES YOU NOTICE OF ACCEPTANCE. BY SUBMITTING
YOUR REGISTRATION FORM, AFFILIATE CERTIFIES THAT YOU HAVE


# Contract Simplification & Risk Prediction (Constitution of India)
**What this notebook adds:**

- Load and flatten a JSON file containing the Constitution (articles & clauses)

- Preprocess text and create datasets for two tasks:
  1. **Summarization (seq2seq)** — fine-tune a Hugging Face summarization model.
  2. **Risk classification** — train a classifier to predict a risk level from article text.

**Notes:**
- This notebook **prepares** the pipeline and includes fully commented functions. It does **not** require you to run long trainings here immediately; run in an environment with GPU (Colab / local with CUDA) for training.
- If your JSON already contains summary targets and risk labels, the notebook will use them. If not, it shows options to generate pseudo-summaries or heuristic risk labels (but manual labeling is recommended for best accuracy).



## 1) Install required packages
Run this cell first in a fresh environment (Colab/local). If you already have `transformers`, `datasets`, and `evaluate`, you can skip installation.


In [ ]:

# !pip install -q transformers datasets evaluate sentencepiece accelerate scikit-learn nltk
# (Uncomment and run the above line in your runtime to install required packages)
print('Installation cell — uncomment the pip line if packages are missing in your environment.')


## 2) Load and flatten the JSON file
This function loads a JSON containing a nested structure of Articles and Clauses and flattens it to a table with columns: `id`, `ArtNo`, `Name`, `text`. It is robust to a few JSON layout variations.


In [ ]:

import json, pandas as pd, os, re
from typing import List, Dict, Any

def load_constitution(json_path: str) -> pd.DataFrame:
    """Load and flatten a Constitution JSON into a DataFrame.

    Args:
        json_path: path to the JSON file (expected to be a list/dict structure similar to the example).

    Returns:
        pd.DataFrame with columns ['id', 'ArtNo', 'Name', 'text'] where each clause/article is one row.
    """
    if not os.path.exists(json_path):
        raise FileNotFoundError(f'JSON file not found: {json_path}')

    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    rows = []

    def recurse(items):
        # items may be list or dict
        if isinstance(items, dict):
            # If dict looks like an article
            art_no = items.get('ArtNo', '')
            name = items.get('Name', '')
            art_desc = items.get('ArtDesc') or items.get('Description') or ''
            if art_desc:
                rows.append({'ArtNo': str(art_no), 'Name': name, 'text': art_desc})
            clauses = items.get('Clauses') or items.get('clauses') or []
            for c in clauses:
                if isinstance(c, dict):
                    clause_no = c.get('ClauseNo', '')
                    clause_desc = c.get('ClauseDesc') or c.get('text') or c.get('clauseDesc') or ''
                    if clause_desc:
                        rows.append({'ArtNo': f"{art_no}.{clause_no}", 'Name': name, 'text': clause_desc})
        elif isinstance(items, list):
            for it in items:
                recurse(it)

    recurse(data)
    df = pd.DataFrame(rows)
    if df.empty:
        raise ValueError('No articles/clauses were found after parsing the JSON — check the JSON structure.')
    df = df.reset_index(drop=True)
    df['id'] = df.index.astype(str)
    # reorder
    df = df[['id', 'ArtNo', 'Name', 'text']]
    return df

# Example usage (update path to your JSON file):
# df = load_constitution('/path/to/constitution.json')
# df.head()
print('Loaded load_constitution function — call it with your JSON file path to get a DataFrame.')


## 3) Preprocessing helpers
Functions to clean text and chunk very long articles (recommended for long seq2seq inputs).


In [ ]:

import nltk
# nltk.download('punkt')  # Uncomment and run once in your environment if punkt is not available
from nltk.tokenize import sent_tokenize


def clean_text(text: str) -> str:
    """Basic cleaning: normalize whitespace and preserve sentence boundaries.

    Args:
        text: raw article/clause text
    Returns:
        cleaned text
    """
    if not isinstance(text, str):
        return ''
    # normalize newlines and whitespace
    t = text.replace('\r', '\n')
    # collapse repeated newlines
    t = re.sub(r"\n{2,}", '\n', t)
    # remove leading/trailing whitespace
    t = t.strip()
    return t


def chunk_text(text: str, max_sentences: int = 8) -> List[str]:
    """Split a long article into smaller chunks by sentences.

    Args:
        text: cleaned text
        max_sentences: maximum sentences per chunk
    Returns:
        list of text chunks
    """
    sents = sent_tokenize(text)
    if len(sents) <= max_sentences:
        return [text]
    chunks = []
    for i in range(0, len(sents), max_sentences):
        chunks.append(' '.join(sents[i:i+max_sentences]))
    return chunks

print('Preprocessing helpers ready (clean_text, chunk_text).')


## 4) Preparing summarization targets
**Important:** Supervised fine-tuning of a summarization model requires *target summaries* for each input. If your JSON already contains gold summaries, the notebook will use them (look for a field named `summary` or similar).

If you don't have gold summaries you have options:
- **Manual annotation** (best quality) — create short summaries for a subset of articles.
- **Pseudo-labeling** — use an existing pretrained summarization model to create initial summaries, then fine-tune on those (a form of self-training). This is quicker but may reinforce existing model biases.

Below we add a helper that will generate pseudo summaries if you have `transformers` available in the runtime.


In [ ]:

def generate_pseudo_summaries(df: pd.DataFrame, model_name: str = 'facebook/bart-large-cnn', batch_size: int = 4):
    """Generate pseudo-summaries using a pretrained summarization model.

    NOTE: this will download a model from Hugging Face and requires internet + transformers installed.
    Use this only if you understand the caveats (pseudo-labels are noisy).

    Returns a new DataFrame with a 'summary' column.
    """
    try:
        from transformers import pipeline
    except Exception as e:
        raise RuntimeError('transformers package required to generate pseudo summaries. Install it and try again.')

    summarizer = pipeline('summarization', model=model_name)
    summaries = []
    for i, row in df.iterrows():
        text = clean_text(row['text'])
        # truncation/safe approach: only send first N characters (adjust for your runtime)
        text_for_model = text[:4000]
        out = summarizer(text_for_model, truncation=True)
        summaries.append(out[0]['summary_text'])
    new_df = df.copy()
    new_df['summary'] = summaries
    return new_df

print('Pseudo-summary helper added (requires transformers pipeline).')


## 5) Risk labeling (heuristic + using provided labels)
If your JSON already includes a `risk` field per article/clause, the notebook will use it. Otherwise a simple heuristic function is provided to create labels (LOW/MEDIUM/HIGH) based on keywords. **Manual labeling is strongly recommended** for production.


In [ ]:

from typing import Literal


def label_risk_heuristic(text: str) -> int:
    """Heuristic risk labeller. Returns 0 (LOW), 1 (MEDIUM), 2 (HIGH).

    This is a heuristic function that looks for keywords often associated with 'risk' (e.g., 'penalty', 'prohibited', 'suspend', 'emergency', 'fine', 'offence').
    It's only a starting point — replace with human labels when possible.
    """
    t = text.lower()
    high = ['emergency', 'suspend', 'penalty', 'offence', 'offence', 'offender', 'criminal', 'liable', 'fined', 'imprison', 'detention']
    medium = ['restriction', 'shall not', 'liable', 'responsible', 'obligation', 'duty', 'penalties']
    score = 0
    for k in high:
        if k in t:
            score += 2
    for k in medium:
        if k in t:
            score += 1
    if score >= 3:
        return 2
    elif score >= 1:
        return 1
    else:
        return 0

print('Heuristic label_risk_heuristic function defined.')


## 6) Build `datasets.Dataset` objects for training
This cell shows how to create HF `datasets` objects from the DataFrame and tokenize them. For summarization we create `input_text` and `summary` columns. For classification we create `text` and `label` columns.


In [ ]:

from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer


def prepare_datasets_for_training(df: pd.DataFrame, tokenizer_name: str = 't5-small', max_input_length: int = 512, max_target_length: int = 128):
    """Prepare HF `datasets` for summarization & classification.

    - Expects df to have columns: 'id', 'text', optionally 'summary', optionally 'risk' or 'label'
    - Returns a dict with 'summarization' and 'classification' Dataset objects (train/valid split applied).
    """
    # Clean text column
    df2 = df.copy()
    df2['text'] = df2['text'].apply(clean_text)

    # Build a summarization dataset if 'summary' exists, otherwise empty
    summ_df = None
    if 'summary' in df2.columns:
        summ_df = df2[['id','text','summary']].rename(columns={'text':'input_text','summary':'target_summary'})
        summ_ds = Dataset.from_pandas(summ_df)
    else:
        summ_ds = None

    # Build classification dataset
    if 'risk' in df2.columns:
        df2['label'] = df2['risk'].astype(int)
    else:
        df2['label'] = df2['text'].apply(label_risk_heuristic)

    cls_df = df2[['id','text','label']]
    cls_ds = Dataset.from_pandas(cls_df)

    # Simple train/validation split
    cls_train_test = cls_ds.train_test_split(test_size=0.1, seed=42)
    datasets_dict = {'classification': cls_train_test}
    if summ_ds is not None:
        summ_train_test = summ_ds.train_test_split(test_size=0.1, seed=42)
        datasets_dict['summarization'] = summ_train_test

    return datasets_dict

print('prepare_datasets_for_training helper created (requires datasets package to run).')


## 7) (Optional) Fine-tune a summarization model (example: T5/BART)
This cell provides a **complete** Seq2Seq training boilerplate using the Hugging Face `Seq2SeqTrainer`. **Do not** run this unless you have GPU + sufficient RAM and `transformers`, `datasets`, and `accelerate` installed.

Key functions are fully documented.


In [ ]:

# NOTE: this cell is an example/training recipe. Run only in a proper runtime (GPU recommended).

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
import numpy as np
import evaluate


def make_tokenizer_and_model(model_name: str = 't5-small'):
    """Load tokenizer and seq2seq model. Change model_name to a larger model if you have resources.

    Returns (tokenizer, model)
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    return tokenizer, model


# Example of how to preprocess for seq2seq

def preprocess_for_summarization(examples, tokenizer, max_input_length=512, max_target_length=128):
    """Tokenize inputs ('input_text') and targets ('target_summary') for seq2seq.
    Returns tokenized dict for Trainer.
    """
    inputs = [clean_text(x) for x in examples['input_text']]
    targets = [x for x in examples['target_summary']]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs['labels'] = labels['input_ids']
    return model_inputs


# compute_metrics example using ROUGE

def compute_rouge_metrics(preds, labels, tokenizer):
    rougelib = evaluate.load('rouge')
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rougelib.compute(predictions=decoded_preds, references=decoded_labels)
    return result


# TRAINING: Example (do not run here unless ready)
# tokenizer, model = make_tokenizer_and_model('t5-small')
# datasets = prepare_datasets_for_training(df_with_summaries, tokenizer_name='t5-small')
# train_dataset = datasets['summarization']['train']
# val_dataset = datasets['summarization']['test']
# data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
# training_args = Seq2SeqTrainingArguments(
#     output_dir='./summarization-checkpoint',
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     predict_with_generate=True,
#     logging_steps=50,
#     evaluation_strategy='steps',
#     eval_steps=200,
#     save_steps=500,
#     num_train_epochs=3,
#     fp16=False
# )
# trainer = Seq2SeqTrainer(model=model, args=training_args, tokenizer=tokenizer, data_collator=data_collator,
#                         train_dataset=train_dataset, eval_dataset=val_dataset)
# trainer.train()

print('Summarization training boilerplate ready (read comments before running).')


## 8) Fine-tune a classifier (example: DistilBERT) for risk prediction
This uses `AutoModelForSequenceClassification` + Trainer. Make sure `datasets`, `transformers` and `evaluate` are installed and run on GPU if possible.


In [ ]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np


def train_risk_classifier(train_dataset, eval_dataset, model_name='distilbert-base-uncased', num_labels=3, output_dir='./risk-classifier'):
    """Fine-tune a transformer-based classifier using Hugging Face Trainer.

    Args:
        train_dataset, eval_dataset: HF datasets (already tokenized)
        model_name: pretrained transformer
        num_labels: number of target classes (e.g., 3 for LOW/MED/HIGH)
        output_dir: where to save the trained model

    Returns:
        trainer object (call trainer.train() to run training)
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

    # A small helper to tokenize the datasets
    def tokenize_fn(examples):
        return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=256)

    tokenized_train = train_dataset.map(tokenize_fn, batched=True)
    tokenized_eval = eval_dataset.map(tokenize_fn, batched=True)

    args = TrainingArguments(
        output_dir=output_dir,
        evaluation_strategy='steps',
        eval_steps=200,
        logging_steps=50,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        save_steps=500,
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_eval,
        tokenizer=tokenizer
    )
    return trainer

print('Classifier training helper defined (Trainer-based).')


## 9) Inference helpers
These functions show how to run the summarization and classification models once they are trained or loaded.


In [ ]:

def summarize_text(text: str, tokenizer, model, device='cpu', max_input_length=1024, max_target_length=150):
    """Generate a summary for `text` using a seq2seq model + tokenizer.

    Returns the generated summary string.
    """
    model.to(device)
    inputs = tokenizer(text, truncation=True, max_length=max_input_length, return_tensors='pt')
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    # generate
    outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=max_target_length, num_beams=4)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary


def predict_risk(text: str, tokenizer, model, device='cpu') -> int:
    """Predict risk label (int) for given text using a sequence classifier.

    Returns an integer label (0/1/2). You can map these to strings externally.
    """
    model.to(device)
    inputs = tokenizer(text, truncation=True, max_length=256, return_tensors='pt')
    inputs = {k:v.to(device) for k,v in inputs.items()}
    with torch.no_grad():
        logits = model(**inputs).logits
    probs = torch.nn.functional.softmax(logits, dim=-1)
    pred = int(torch.argmax(probs, dim=-1).cpu().numpy()[0])
    return pred

print('Inference helper functions added.')


## 10) Example workflow (what to run, in order)
1. Install dependencies (pip) and ensure GPU is available for training.
2. Place your constitution JSON somewhere accessible (e.g., `/content/constitution.json` in Colab).
3. Run the **Load** cell: `df = load_constitution('/path/to/constitution.json')`.
4. (Optional) Create pseudo summaries: `df_with_summ = generate_pseudo_summaries(df)` — only if you don't have gold summaries.
5. Prepare datasets: `datasets = prepare_datasets_for_training(df_with_summ_or_df)`.
6. For summarization: follow the Seq2SeqTrainer boilerplate to train.
7. For risk classifier: use `train_risk_classifier()` helper to get a Trainer and run `trainer.train()`.
8. Run inference using `summarize_text()` and `predict_risk()`.

Below is a small code snippet showing these steps programmatically (edit paths and model names as you need).


In [ ]:

# Example (do not run verbatim unless paths/models exist in your environment):
# json_path = '/path/to/constitution.json'
# df = load_constitution(json_path)
# # If you have no summaries, create pseudo: df = generate_pseudo_summaries(df)
# datasets = prepare_datasets_for_training(df, tokenizer_name='t5-small')
# # For classification
# cls_train = datasets['classification']['train']
# cls_eval = datasets['classification']['test']
# trainer = train_risk_classifier(cls_train, cls_eval, model_name='distilbert-base-uncased')
# trainer.train()
# # For summarization: follow the Seq2Seq cell to construct a trainer and train
print('Example workflow cell: edit the paths and run in your environment.')


---

### Final notes & recommendations
- **Manual labeling:** For risk prediction, create a labeled dataset (even 200-500 human-labeled examples) — heuristics are a start but human labels give far better results.
- **Summaries:** Gold summaries are best. If you must pseudo-label, clean and spot-check generated summaries.
- **Evaluation:** Use `rouge` for summarization and `accuracy`, `f1` for classification. Validate on holdout sets.
- **Deployment:** For inference-only use, you can host the summarization and classifier models (Hugging Face Hub or your own endpoints) and call them via REST.

If you want, I can now:
- Put together a smaller runnable demo (that uses a tiny model and trains for 1 epoch on a subset) inside this notebook so you can run it even on CPU for testing.
- Or modify the heuristic rules / labeling to match your exact risk definitions.

Tell me which of the two you'd like next, and I will add the demo cell right into the notebook.
